# Goal:

For the sales, I'd like to denote whether it was sold at a loss or a gain with a red or green icon. The rest of the info could be put into a pop up box, if that works

oh and btw maybe put listings in blue or something so we can see that they are not sold yet

# Imports

In [15]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup
import gspread

## Data read-in

In [16]:
df = pd.read_csv('Prager map details - final.csv')

In [17]:
# # Drop the 'URL' column if you no longer need it
# df = df.drop(columns=['URL'])

In [18]:
# df['Description'].iloc[0]

## Geocode

In [19]:
df.columns

Index(['#', 'Street', 'City', 'County', 'State', 'Lawsuit/foreclosure',
       'Landlord Lender', 'A-1', 'A-2', 'B', 'Landlord Loan', 'GL Lender',
       'A-1.1', 'A-2.1', 'B.1', 'GL Lender 2', 'Unnamed: 16',
       'Ground Lease Loan', 'Complex name'],
      dtype='object')

In [20]:
df['Full_Address'] = df['#'] + ' ' + df['Street'] + ' ' + df['City'] + ' ' + df['County'] + ' ' + df['State']

In [21]:
df['Display_Address'] = df['#'] + ' ' + df['Street'] + ' ' + df['City'] + ', ' + df['State']

In [22]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [23]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'Full_Address' column and store the results in the 'geocoded' column
df['geocoded'] = df['Full_Address'].apply(geocode)

# Split 'geocoded' into 'lat' and 'lon', handling None values
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x is None else x).apply(pd.Series)

# Convert 'lat' and 'lon' to floats
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

# Create the 'geometry' column with Point objects
df['geometry'] = df.apply(lambda row: Point(row['lon'], row['lat']) if pd.notnull(row['lat']) and pd.notnull(row['lon']) else None, axis=1)

# Drop rows where geometry is None
df = df.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set CRS (Coordinate Reference System) to WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)


/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,#,Street,City,County,State,Lawsuit/foreclosure,Landlord Lender,A-1,A-2,B,...,GL Lender 2,Unnamed: 16,Ground Lease Loan,Complex name,Full_Address,Display_Address,geocoded,lat,lon,geometry
0,333,City Blvd W,Orange County,Orange,CA,NaN,Technology FCU,75.750,NaN,NaN,...,NaN,NaN,0.000,NaN,333 City Blvd W Orange County Orange CA,"333 City Blvd W Orange County, CA","(33.7870982, -117.8936947)",33.787098,-117.893695,POINT (-117.89369 33.78710)
1,"300, 400, 801, 901",Int'l Pwy (+ 1001 Heathrow Park Lane),Lake Mary,Seminole,FL,Y,UMB,70.000,NaN,18.000,...,NaN,NaN,79.700,NaN,"300, 400, 801, 901 Int'l Pwy (+ 1001 Heathrow ...","300, 400, 801, 901 Int'l Pwy (+ 1001 Heathrow ...","(28.7801958, -81.356084)",28.780196,-81.356084,POINT (-81.35608 28.78020)
2,1500 & 1600,Riveredge Pwy,Atlanta,Fulton,GA,NaN,UMB,38.000,NaN,8.500,...,NaN,NaN,43.000,NaN,1500 & 1600 Riveredge Pwy Atlanta Fulton GA,"1500 & 1600 Riveredge Pwy Atlanta, GA","(33.9081754, -84.4362621)",33.908175,-84.436262,POINT (-84.43626 33.90818)
3,263,Shuman Blvd,Naperville,DuPage,IL,NaN,UMB,27.000,NaN,8.500,...,NaN,NaN,45.430,NaN,263 Shuman Blvd Naperville DuPage IL,"263 Shuman Blvd Naperville, IL","(41.8043611, -88.1514825)",41.804361,-88.151483,POINT (-88.15148 41.80436)
4,500-540,Lake Cook Road,Deerfield,Lake,IL,Y,UMB,72.000,NaN,19.000,...,NaN,NaN,104.000,NaN,500-540 Lake Cook Road Deerfield Lake IL,"500-540 Lake Cook Road Deerfield, IL","(42.1537689, -87.8406607)",42.153769,-87.840661,POINT (-87.84066 42.15377)
5,3025,Highland,Downers Grove,DuPage,IL,NaN,UMB,23.000,NaN,5.000,...,NaN,NaN,32.250,NaN,3025 Highland Downers Grove DuPage IL,"3025 Highland Downers Grove, IL","(41.8330351, -88.0056721)",41.833035,-88.005672,POINT (-88.00567 41.83304)
6,35,W Wacker Dr,Chicago,Cooks,IL,NaN,UMB,171.500,NaN,45.500,...,NaN,NaN,220.000,NaN,35 W Wacker Dr Chicago Cooks IL,"35 W Wacker Dr Chicago, IL","(41.8857787, -87.6294925)",41.885779,-87.629492,POINT (-87.62949 41.88578)
7,5600,83rd St W,Bloomington,Hennepin,MN,NaN,UMB,20.064,7.315,5.326,...,NaN,NaN,32.400,Normandale Office Park,5600 83rd St W Bloomington Hennepin MN,"5600 83rd St W Bloomington, MN","(44.8545062, -93.35533819999999)",44.854506,-93.355338,POINT (-93.35534 44.85451)
8,8000,Norman Center Dr,Bloomington,Hennepin,MN,Y,UMB,21.247,7.746,5.640,...,NaN,NaN,41.340,Normandale Office Park,8000 Norman Center Dr Bloomington Hennepin MN,"8000 Norman Center Dr Bloomington, MN","(44.8522914, -93.3510984)",44.852291,-93.351098,POINT (-93.35110 44.85229)
9,8300,Norman Center Dr,Bloomington,Hennepin,MN,NaN,UMB,18.037,6.576,4.788,...,NaN,NaN,36.758,Normandale Office Park,8300 Norman Center Dr Bloomington Hennepin MN,"8300 Norman Center Dr Bloomington, MN","(44.8526766, -93.35298209999999)",44.852677,-93.352982,POINT (-93.35298 44.85268)


In [24]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [25]:
gdf.to_file('data.geojson', driver='GeoJSON')

In [26]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Prager_Map_10_30_24


In [27]:
df.columns

Index(['#', 'Street', 'City', 'County', 'State', 'Lawsuit/foreclosure',
       'Landlord Lender', 'A-1', 'A-2', 'B', 'Landlord Loan', 'GL Lender',
       'A-1.1', 'A-2.1', 'B.1', 'GL Lender 2', 'Unnamed: 16',
       'Ground Lease Loan', 'Complex name', 'Full_Address', 'Display_Address',
       'geocoded', 'lat', 'lon', 'geometry'],
      dtype='object')

<a href="https://therealdeal.com/chicago/2024/04/03/david-gassman-sells-chicago-apartment-portfolio-for-72m/"> 6954 North Sheridan Road, Chicago</a>

In [28]:
gdf.columns

Index(['#', 'Street', 'City', 'County', 'State', 'Lawsuit/foreclosure',
       'Landlord Lender', 'A-1', 'A-2', 'B', 'Landlord Loan', 'GL Lender',
       'A-1.1', 'A-2.1', 'B.1', 'GL Lender 2', 'Unnamed: 16',
       'Ground Lease Loan', 'Complex name', 'Full_Address', 'Display_Address',
       'geocoded', 'lat', 'lon', 'geometry'],
      dtype='object')